# Loan Eligiblity Prediction 🤝📜📃⚖️💵

### **Motive** :- 

---




> In today’s world of recession and uncertain
economy following the pandemic, it is vital to have
healthy and sustainable financing no matter who
you are. A big part in ensuring this is to stay clear of
risky loan traps and prevent taking on
unsustainable debts.
A smart individual would diligently measure the
amount and purpose of each loan against their own
specific information to determine their *ability* to
pay the loan back. There is certainly a large scope
for programs and algorithms that can help in
making this decision with high accuracy.



---





#### Explanation Manual :- 
The code imports several libraries that are commonly used for machine learning tasks, such as:





*   pickle for saving model
*   numpy for numerical computations
*   pandas for data manipulation and analysis
*   RandomForestClassifier from sklearn.ensemble for building Random Forest classification models
*   classification_report from sklearn.metrics for evaluating the performance of the model by computing the accuracy score.
*   train_test_split from sklearn.model_selection for splitting the data into training and testing sets.

In [2]:
#Importing important Library's

import pickle
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [166]:
# Reading DataSet and converting it to dataframe using pandas
df = pd.read_csv("/content/drive/MyDrive/Hackathon_IITII/Juputer_Files/Track_1.csv")

In [167]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 23 columns):
 #   Column                                         Non-Null Count  Dtype 
---  ------                                         --------------  ----- 
 0   Unnamed: 0                                     600 non-null    int64 
 1   Number of people who will provide maintenance  600 non-null    int64 
 2   Loan History                                   600 non-null    object
 3   Purpose of taking loan                         591 non-null    object
 4   loan amount taken                              600 non-null    int64 
 5   Guarantor or Debtor                            600 non-null    object
 6   Number of years of employment                  447 non-null    object
 7   Marital Status                                 600 non-null    object
 8   Number of loans taken from current bank        600 non-null    int64 
 9   Age of the applicant in Number of Years        600 non-null    in

## **Preprocessing Data**



> Data preprocessing, a component of data preparation, describes any type of processing performed on raw data to prepare it for another data processing procedure.

In our case we will be dropping, Renaiming few coloumns the conculsion were made on the basis of EDA Notebook Check it out for reference :- [EDA_OF_DATA](https://colab.research.google.com/drive/1IxWLBCwm9BM5IPqQAxSoVxbm7OmoKqYk?usp=sharing)







In [168]:
# Droping the Column which are not useful

df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df = df.drop(['1=defaulted','Other loans plans taken'],axis=1)

In [169]:
#Renameing Coloumn Name For Convinence

df.rename(columns = {'Number of people who will provide maintenance':'NPPM','Purpose of taking loan': 'Objective',
                     'loan amount taken': 'Amount','Guarantor or Debtor': 'Guarantor','Loan History':'LoanStatus',
                     'Number of years of employment': 'Experience','Marital Status' : 'M_Status','Number of loans taken from current bank': 'ExistingLoan',
                     'Age of the applicant in Number of Years': 'Age','amount in current account': 'CA_Balance',
                     'amount in savings account': 'SA_Balance','% of income paid as installment': 'PI_Payment',
                     'Working abroad or not': 'WorkAB', 'Is there telephone number available': 'PhN',
                     'time duration for loan': 'Tenure', 'Owned property': 'prop', 'Type of job performed': 'JobTyp',
                     'Type of Housing':'HouseT','Number of years of stay in current address': 'NOE','Loan Defaulted or not': 'prediction','Other loans plans taken' : 'olpt'}, inplace = True)

In [170]:
#Replacing nan values with single space so they can be treated as entity

df['Objective'] = df['Objective'].replace(np.nan, '', regex=True)
df['Experience'] = df['Experience'].replace(np.nan, '', regex=True)

In [171]:
## Let's Check Out For Null Values In Data Frame

df.isna().sum()

NPPM            0
LoanStatus      0
Objective       0
Amount          0
Guarantor       0
Experience      0
M_Status        0
ExistingLoan    0
Age             0
CA_Balance      0
SA_Balance      0
PI_Payment      0
WorkAB          0
PhN             0
Tenure          0
prop            0
JobTyp          0
HouseT          0
NOE             0
prediction      0
dtype: int64

In [172]:
df.head()

,NPPM,LoanStatus,Objective,Amount,Guarantor,Experience,M_Status,ExistingLoan,Age,CA_Balance,SA_Balance,PI_Payment,WorkAB,PhN,Tenure,prop,JobTyp,HouseT,NOE,prediction
0,1,critical account/other loans existing (not at ...,New Car Purchase,1028,none,between 1 and 4 years,female and divorced/seperated/married,2,36,no current account,less than 100,4,Yes,No,18,Real Estate,skilled employee / official,own,3,1
1,1,existing loans paid back duly till now,Purchase of radio/television,10222,none,greater than 4 years,male and single,1,37,no current account,no savings account,4,Yes,Yes,48,car or other property,skilled employee / official,own,3,1
2,1,existing loans paid back duly till now,Purchase of radio/television,701,none,between 1 and 4 years,male and married/widowed,1,40,less than 0,less than 100,4,Yes,No,12,Real Estate,unskilled - resident,own,2,1
3,1,existing loans paid back duly till now,Purchase of radio/television,1670,none,less than a year,female and divorced/seperated/married,1,22,between 0 and 200,less than 100,4,Yes,Yes,9,car or other property,skilled employee / official,own,2,2
4,1,critical account/other loans existing (not at ...,New Car Purchase,1163,none,between 1 and 4 years,male and single,1,44,no current account,between 500 and 1000,4,Yes,Yes,12,Real Estate,skilled employee / official,own,4,1


In [6]:
def preprocessing(df):
    df.Objective = df.Objective.replace({"New Car Purchase": 0, "Purchase of radio/television" : 1,'Purchase of furniture/equipment' : 2,'Old Car Repair': 3,'Education Loan': 4,
                                        'Loan for business establishment': 5,'Other repairs': 6,'Purchase of domestic appliances': 7,'Loan for retraining': 8,'': 9,'nan':9})

    df.Guarantor = df.Guarantor.replace({'none' : 0,'gaurantor' : 1,'co-applicant':  2,'': 3})

    df.LoanStatus = df.LoanStatus.replace({'critical account/other loans existing (not at this bank)' : 0,'existing loans paid back duly till now' : 1,'all loans at this bank paid back duly':  2,'delay in paying off loans in the past' : 3,
                                           'no loans taken/all loans paid back duly': 4,'': 5,})

    df.Experience = df.Experience.replace({'between 1 and 4 years': 0, 'greater than 4 years': 1,'less than a year': 2,'unemployed': 3,'':4})

    df.M_Status = df.M_Status.replace({'female and divorced/seperated/married': 0,'male and single': 1,'male and married/widowed': 2,
                                      'male and divorced/seperated': 3,'':4})

    df.CA_Balance = df.CA_Balance.replace({'no current account': 0,'less than 0': 1,'between 0 and 200': 3,'greater than 200': 4,'':5})

    df.SA_Balance = df.SA_Balance.replace({'less than 100': 0,'no savings account': 1,'between 500 and 1000': 2,
                                          'between 100 and 500': 3,'greater than 1000': 4,'':5})

    df.WorkAB = df.WorkAB.replace({'Yes': 1,'No': 0,'':2})

    df.PhN = df.PhN.replace({'Yes': 1,'No': 0,'':2})

    df.prop = df.prop.replace({'Real Estate': 0,'car or other property': 1,'No property': 2,'building society savings agreement/life insurance': 3,'':4})

    df.JobTyp = df.JobTyp.replace({'skilled employee / official': 0,'unskilled - resident': 1,'unemployed/ unskilled - non-resident': 2,'management/ self-employed/highly qualified employee/ officer': 3,'':4})

    df.HouseT = df.HouseT.replace({'own': 0,'for free': 1,'rent': 3,'':4})

    df.prediction = df.prediction.replace({2 : 0})

    # df.olpt = df.olpt.replace({'none' : 0,'stores': 1,'bank': 2})
    
    return df

df = preprocessing(df)

In [175]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 20 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   NPPM          600 non-null    int64
 1   LoanStatus    600 non-null    int64
 2   Objective     600 non-null    int64
 3   Amount        600 non-null    int64
 4   Guarantor     600 non-null    int64
 5   Experience    600 non-null    int64
 6   M_Status      600 non-null    int64
 7   ExistingLoan  600 non-null    int64
 8   Age           600 non-null    int64
 9   CA_Balance    600 non-null    int64
 10  SA_Balance    600 non-null    int64
 11  PI_Payment    600 non-null    int64
 12  WorkAB        600 non-null    int64
 13  PhN           600 non-null    int64
 14  Tenure        600 non-null    int64
 15  prop          600 non-null    int64
 16  JobTyp        600 non-null    int64
 17  HouseT        600 non-null    int64
 18  NOE           600 non-null    int64
 19  prediction    600 non-null   

In [176]:
## Spliting Our Data Frame in X,Y with our features going in the x and our target/result going in the y variable

X = df.drop(['prediction'],axis=1)
Y = df['prediction']

In [177]:
#Spliting Our Data [ X(features) ,Y(result/prediction)] For Training and Testing Purposes. 
#The split will be int the ratio of 80:20.

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [178]:
print("Size of X Train:- " + str(X_train.shape))
print("Size of X Test:- " + str(X_test.shape))
print("Size of Y Train:- " + str(y_train.shape))
print("Size of Y Test:- " + str(y_test.shape))

Size of X Train:- (480, 19)
Size of X Test:- (120, 19)
Size of Y Train:- (480,)
Size of Y Test:- (120,)


# **Building our Machine Learning Model**

We will be using random forest for the given data as we know it will be giving us best accuracy this was concluded from accuracy Notebook :- [AccuracyNotebook](https://googel.com). First we will be building model on given dataset. Then we will SMOT over it to imporove accuracy.

In [179]:
reg_rf = RandomForestClassifier(max_depth=180,min_samples_split= 5,n_estimators= 152)
reg_rf.fit(X_train, y_train)

RandomForestClassifier(max_depth=180, min_samples_split=5, n_estimators=152)

In [180]:
y_pred_1 = reg_rf.predict(X_test)

In [182]:
print(metrics.classification_report(y_test, y_pred_1))

              precision    recall  f1-score   support

           0       0.81      0.37      0.51        35
           1       0.79      0.96      0.87        85

    accuracy                           0.79       120
   macro avg       0.80      0.67      0.69       120
weighted avg       0.80      0.79      0.76       120



In [183]:
print(f'Accuracy: {accuracy_score(y_test, y_pred_1) * 100}')

Accuracy: 79.16666666666666


# Oversampling Minority Class
Applying oversampling technique. For Improving F1 Score and Accuracy of The Model. we will be adding a bit more data then the majority class because oversampling might have some overlapping or dupicate data so we will add a bit more data then needed so accuracy and F1 score will be balanced.

In [184]:
# Class count
count_class_1, count_class_0 = df.prediction.value_counts()

# Divide by class
df_class_0 = df[df['prediction'] == 0]
df_class_1 = df[df['prediction'] == 1]

In [185]:
df['prediction'].value_counts()

1    422
0    178
Name: prediction, dtype: int64

In [186]:
# Class count
count_class_1, count_class_0 = df.prediction.value_counts()

# Divide by class
df_class_0 = df[df['prediction'] == 0]
df_class_1 = df[df['prediction'] == 1]

In [187]:
count_class_1, count_class_0

(422, 178)

In [188]:
df_class_0_over = df_class_0.sample(count_class_1+100,replace=True)
df_test_over = pd.concat([df_class_1,df_class_0_over],axis=0)

print('Random over-sampling:')
print(df_test_over.prediction.value_counts())

Random over-sampling:
0    522
1    422
Name: prediction, dtype: int64


In [189]:
X_m2 = df_test_over.drop('prediction',axis='columns')
y_m2 = df_test_over['prediction']

from sklearn.model_selection import train_test_split
X_m2_train, X_m2_test, y_m2_train, y_m2_test = train_test_split(X_m2, y_m2, test_size=0.2, random_state=15, stratify=y_m2)

In [190]:
y_m2_train.value_counts()

0    417
1    338
Name: prediction, dtype: int64

# **Building our Machine Learning Model**

We will be using random forest for the given data as we know it will be giving us best accuracy this was concluded from accuracy Notebook :- [AccuracyNotebook]. First we will be building model on given dataset. Then we will SMOT over it to imporove accuracy.

In [198]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

reg_rf_1 = RandomForestClassifier(max_depth=180,min_samples_split= 5,n_estimators= 152)
reg_rf_1.fit(X_m2_train, y_m2_train)
ym_pred = reg_rf_1.predict(X_m2_test)
print(metrics.classification_report(y_m2_test, ym_pred))

              precision    recall  f1-score   support

           0       0.91      0.97      0.94       105
           1       0.96      0.88      0.92        84

    accuracy                           0.93       189
   macro avg       0.94      0.93      0.93       189
weighted avg       0.93      0.93      0.93       189



In [199]:
print(f'Accuracy: {accuracy_score(y_m2_test, ym_pred)*100}')

Accuracy: 93.12169312169311


In [202]:
# Save the trained model as a pickle string.

pickle.dump(reg_rf_1, open('model_final.pkl','wb'))